In [8]:
%load_ext autoreload
%autoreload 2

import plotly.express as px
import plotly.graph_objects as go
import re
import time

import utils
from slurm import SlurmClient

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Slurm Queue

In [72]:
!sinfo

PARTITION AVAIL  TIMELIMIT  NODES  STATE NODELIST
q1*          up 7-00:00:00      1   unk* h100-st-p548xlarge-4
q1*          up 7-00:00:00     28    mix h100-st-p548xlarge-[7,9-10,13-14,29,38,48-49,227-242,333,448-449]
q1*          up 7-00:00:00    312  alloc h100-st-p548xlarge-[0-3,5-6,11-12,15-28,30-37,41-47,79-197,217-221,264-305,325-332,334-344,350-353,358-443]
q1*          up 7-00:00:00    109   idle h100-st-p548xlarge-[8,39-40,50-78,198-216,222-226,243-263,306-324,345-349,354-357,444-447]


In [73]:
sl = SlurmClient()
sl.get_queue_summary()

,ACCOUNT,ST,NODES
0,all,R,17
1,ar-ai-hipri,R,267
2,ar-ai-midpri,R,2
3,ar-ai-research-interns,R,3
4,ar-ai-voice-hipri,PD,92
5,ar-ai-voice-hipri,R,53


In [74]:
sl.get_recent_queue('all')

,JOBID,ACCOUNT,NAME,USER,ST,START_TIME,TIME,NODES,TIME_SECS
46,7765,all,sft-eval,jiuhai,R,2024-08-08T23:30:57,1:51:28,16,6688
47,7761,all,sft-eval,jiuhai,R,2024-08-08T23:00:05,2:22:20,1,8540


In [75]:
sl.get_recent_queue('ar-ai-research-interns')

,JOBID,ACCOUNT,NAME,USER,ST,START_TIME,TIME,NODES,TIME_SECS
51,7489,ar-ai-research-interns,bash,yfzhuang,R,2024-08-08T18:05:03,7:17:24,1,26244
52,7488,ar-ai-research-interns,bash,yfzhuang,R,2024-08-08T18:03:36,7:18:51,1,26331
50,7337,ar-ai-research-interns,jupyter,imzyc,R,2024-08-08T00:14:40,1-01:07:47,1,90467


In [76]:
sl.get_recent_queue('ar-ai-voice-hipri')

,JOBID,ACCOUNT,NAME,USER,ST,START_TIME,TIME,NODES,TIME_SECS
65,7375,ar-ai-voice-hipri,21CC-MAIN-2023-14,arashe,R,2024-08-09T01:11:13,11:15,2,675
66,7374,ar-ai-voice-hipri,20CC-MAIN-2023-14,arashe,R,2024-08-09T00:27:33,54:55,2,3295
69,7373,ar-ai-voice-hipri,19CC-MAIN-2023-14,arashe,R,2024-08-08T21:10:30,4:11:58,2,15118
70,7372,ar-ai-voice-hipri,18CC-MAIN-2023-14,arashe,R,2024-08-08T21:02:22,4:20:06,2,15606
71,7371,ar-ai-voice-hipri,17CC-MAIN-2023-14,arashe,R,2024-08-08T20:59:07,4:23:21,2,15801
72,7370,ar-ai-voice-hipri,16CC-MAIN-2023-14,arashe,R,2024-08-08T20:58:24,4:24:04,2,15844
73,7368,ar-ai-voice-hipri,14CC-MAIN-2023-14,arashe,R,2024-08-08T20:45:02,4:37:26,2,16646
75,7367,ar-ai-voice-hipri,13CC-MAIN-2023-14,arashe,R,2024-08-08T20:35:16,4:47:12,2,17232
76,7366,ar-ai-voice-hipri,12CC-MAIN-2023-14,arashe,R,2024-08-08T19:47:03,5:35:25,2,20125
77,7365,ar-ai-voice-hipri,11CC-MAIN-2023-14,arashe,R,2024-08-08T19:38:59,5:43:29,2,20609


In [54]:
q = sl.get_queue_summary()
running = {}
for _, row in q.iterrows():
    if row.ST == 'R':
        running[row.ACCOUNT] = row.NODES
print(running)

quota = {'ar-ai-research-interns': 8,
         'ar-ai-voice-hipri': 50, 'ar-ai-hipri': 400}

evict_ids = {}
for acc in running:
    if acc not in quota:  # quota = 0
        print(acc)

{'all': 17, 'ar-ai-hipri': 274, 'ar-ai-midpri': 2, 'ar-ai-research-interns': 3, 'ar-ai-voice-hipri': 53}
all
ar-ai-midpri


In [30]:
q_summary = q.groupby(['ACCOUNT', 'ST']).aggregate(
    {'NODES': 'sum'}).reset_index()
q_summary

,ACCOUNT,ST,NODES
0,all,R,17
1,ar-ai-hipri,R,280
2,ar-ai-midpri,R,2
3,ar-ai-research-interns,R,3
4,ar-ai-voice-hipri,PD,94
5,ar-ai-voice-hipri,R,53


# Metrics

# Launch